## Pease methodology

In [1]:
import pandas as pd
import nltk
import glob
import os
import time
path = '/Users/Svesketerning/Google-Drev/experiments'
timestr = time.strftime("%Y%m%d-%H%M")

In [3]:
# Definining word_extract function - will be copied for other experiments.
def word_extract(dataframe_context,focus_words):
    result = []
    contexts = [x for x in dataframe_context.columns]
    for index in range(len(dataframe_context.index)):
        for c in contexts:
            sentences = nltk.sent_tokenize(dataframe_context[c][index])
            for sent in sentences:
                sent = sent.lower()
                overlap = [x for x in focus_words if x in sent]
                if len(overlap)>0:
                    arc_id = df_contexts['id'][index]
                    result.append({'id':arc_id, # NOTE THAT YOU HAVE TO LOAD ARXIV DATABASE FILE AS df_arxiv!!
                                   'mainmathcat': df_arxiv.loc[df_arxiv['id'] == arc_id]['mainmathcat'].item(),
                                   'context': c, 'sentence': sent, 'focuswords': ",".join(overlap)})
    df = pd.concat([pd.DataFrame(result[i], index=['id']) for i in range(len(result))])
    df = df.set_index('id')
    return df 

In [2]:
# Load latest context and ArXiv database file
LatestContextFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments/database-files/arxiv_contexts*'),key=os.path.getctime)
df_contexts = pd.read_feather(LatestContextFile)

LatestDatabaseFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments/database-files/arxiv_extended_database*'),key=os.path.getctime)
df_arxiv = pd.read_feather(LatestDatabaseFile)

In [33]:
# Taken form Pease et al. 2019, p.12
pease_words = ['since','because',', as','thus','therefore',', so','expla','underst']
pease_pct = [0.73,0.92,0.71,0.62,0.45,0.7,1,0.82]

In [31]:
pease_df = word_extract(df_contexts,pease_words)
pease_df = pease_df.reset_index()
pease_df.to_feather(path+'/pease-data/pease_fullraw'+timestr+'.feather')

In [4]:
# Load latest pease explain database
LatestPeaseExplainFile = max(glob.iglob('/Users/Svesketerning/Google-Drev/experiments//pease-data/explain_peasecontext_freq*'),key=os.path.getctime)

explain_peasecontext_freq = pd.read_feather(LatestPeaseExplainFile)


#####  Loop counts explanations per million words over context and adds total.

In [85]:
contexts = ['outer','theorem', 'meta', 'proof', 'other']
values_context = []
total_expla = 0
total_words = 0
for j in contexts:
    a = 0
    result = []
    for i in pease_words:
        c = len(pease_df.loc[(pease_df['context'] == j) & (pease_df['focuswords'] == i)])
        a += c*pease_pct[pease_words.index(i)]
    b = df_arxiv[j].sum()
    total_words += b
    total_expla += a 
    values_context.append(a/b * 10**6)
result.append({'outer': values_context[0],'theorem': values_context[1],
               'meta': values_context[2], 'proof': values_context[3], 
               'other': values_context[4], 'total': (total_expla/total_words * 10**6)})
explain_peasecontext_freq = pd.concat([pd.DataFrame(result[i], index=[0]) for i in range(len(result))])

In [7]:
# Saves to Pease-data and tex folder
explain_peasecontext_freq.to_feather(path+'/pease-data/explain_peasecontext_freq'+timestr+'.feather')
explain_peasecontext_freq.round(2).to_latex(path+'/tex/explain_peasecontext_freq'+timestr+'.tex')

,outer,theorem,meta,proof,other,total
0,2790.7,633.1,4060.17,7888.26,851.33,3738.36
